# Store Sales

## Problem Statement

Corporación Favorita, an Ecuadorian grocery retailer, faces challenges in accurately forecasting unit sales across its stores, leading to overstocking or understocking and resulting in increased food waste and decreased customer satisfaction. To address this, we will develop a machine learning model to predict daily unit sales at less than 0.5 RMSE by average. Utilizing historical sales data, store metadata, promotional information, oil prices, and holiday events, we aim to deliver a robust forecasting model within 4 weeks.

## Imports

In [175]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler

In [176]:
train = pd.read_csv('data/train_combined.csv')
test = pd.read_csv('data/test_combined.csv')

/var/folders/xk/5gx1tq55563djqn2f91dfp040000gn/T/ipykernel_31234/3076599388.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('data/train_combined.csv')


In [177]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [178]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 17 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Unnamed: 0           int64         
 1   id                   int64         
 2   date                 datetime64[ns]
 3   store_nbr            int64         
 4   family               object        
 5   sales                float64       
 6   onpromotion          int64         
 7   daily_oil_price      float64       
 8   holiday_type         object        
 9   holiday_locale       object        
 10  locale_name          object        
 11  holiday_description  object        
 12  holiday_transferred  object        
 13  city                 object        
 14  state                object        
 15  store_type           object        
 16  store_cluster        int64         
dtypes: datetime64[ns](1), float64(2), int64(5), object(9)
memory usage: 396.1+ MB


In [179]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Unnamed: 0           28512 non-null  int64         
 1   id                   28512 non-null  int64         
 2   date                 28512 non-null  datetime64[ns]
 3   store_nbr            28512 non-null  int64         
 4   family               28512 non-null  object        
 5   onpromotion          28512 non-null  int64         
 6   daily_oil_price      28512 non-null  float64       
 7   holiday_type         28512 non-null  object        
 8   holiday_locale       28512 non-null  object        
 9   locale_name          28512 non-null  object        
 10  holiday_description  28512 non-null  object        
 11  holiday_transferred  28512 non-null  object        
 12  city                 28512 non-null  object        
 13  state                28512 non-

## Data Preprocessing

In [180]:
# Add an indicator to distinguish between train and test
train['dataset'] = 'train'
test['dataset'] = 'test'

# Combine the datasets
combined = pd.concat([train, test], axis=0)

### Check for missing and unexpected values 

In [181]:
combined.isnull().sum()

Unnamed: 0                 0
id                         0
date                       0
store_nbr                  0
family                     0
sales                  28512
onpromotion                0
daily_oil_price            0
holiday_type               0
holiday_locale             0
locale_name                0
holiday_description        0
holiday_transferred        0
city                       0
state                      0
store_type                 0
store_cluster              0
dataset                    0
dtype: int64

In [182]:
# Drop holiday_description since will not use for modeling
combined.drop(columns=['holiday_description'], inplace=True)

In [183]:
categorical_columns = ['family', 'holiday_type', 'holiday_locale', 'locale_name', 'holiday_transferred', 'city', 'state', 'store_type', 'store_cluster']

In [184]:
# Check unique values
for col in categorical_columns:
    print(f"{col} unique values: ", combined[col].unique())

family unique values:  ['AUTOMOTIVE' 'BABY CARE' 'BEAUTY' 'BEVERAGES' 'BOOKS' 'BREAD/BAKERY'
 'CELEBRATION' 'CLEANING' 'DAIRY' 'DELI' 'EGGS' 'FROZEN FOODS' 'GROCERY I'
 'GROCERY II' 'HARDWARE' 'HOME AND KITCHEN I' 'HOME AND KITCHEN II'
 'HOME APPLIANCES' 'HOME CARE' 'LADIESWEAR' 'LAWN AND GARDEN' 'LINGERIE'
 'LIQUOR,WINE,BEER' 'MAGAZINES' 'MEATS' 'PERSONAL CARE' 'PET SUPPLIES'
 'PLAYERS AND ELECTRONICS' 'POULTRY' 'PREPARED FOODS' 'PRODUCE'
 'SCHOOL AND OFFICE SUPPLIES' 'SEAFOOD']
holiday_type unique values:  ['Holiday' 'none' 'Work Day' 'Additional' 'Event' 'Transfer' 'Bridge']
holiday_locale unique values:  ['National' 'none' 'Local' 'Regional']
locale_name unique values:  ['Ecuador' 'none' 'Manta' 'Cotopaxi' 'Cuenca' 'Libertad' 'Riobamba' 'Puyo'
 'Guaranda' 'Imbabura' 'Machala' 'Latacunga' 'El Carmen' 'Santo Domingo'
 'Cayambe' 'Guayaquil' 'Esmeraldas' 'Ambato' 'Ibarra' 'Quevedo'
 'Santo Domingo de los Tsachilas' 'Santa Elena' 'Quito' 'Loja' 'Salinas']
holiday_transferred unique valu

In [185]:
combined['holiday_transferred'].replace({False: 'False'}, inplace=True)

/var/folders/xk/5gx1tq55563djqn2f91dfp040000gn/T/ipykernel_31234/3342495674.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined['holiday_transferred'].replace({False: 'False'}, inplace=True)


In [186]:
combined['holiday_transferred'].unique()

array(['False', 'none', 'True'], dtype=object)

### Feature Enegineering

In [187]:
# Date-based features
combined['day_of_week'] = combined['date'].dt.dayofweek
combined['month'] = combined['date'].dt.month
combined['quarter'] = combined['date'].dt.quarter

In [188]:
combined.head()

,Unnamed: 0,id,date,store_nbr,family,sales,onpromotion,daily_oil_price,holiday_type,holiday_locale,locale_name,holiday_transferred,city,state,store_type,store_cluster,dataset,day_of_week,month,quarter
0,0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,Holiday,National,Ecuador,False,Quito,Pichincha,D,13,train,1,1,1
1,1,1,2013-01-01,1,BABY CARE,0.0,0,93.14,Holiday,National,Ecuador,False,Quito,Pichincha,D,13,train,1,1,1
2,2,2,2013-01-01,1,BEAUTY,0.0,0,93.14,Holiday,National,Ecuador,False,Quito,Pichincha,D,13,train,1,1,1
3,3,3,2013-01-01,1,BEVERAGES,0.0,0,93.14,Holiday,National,Ecuador,False,Quito,Pichincha,D,13,train,1,1,1
4,4,4,2013-01-01,1,BOOKS,0.0,0,93.14,Holiday,National,Ecuador,False,Quito,Pichincha,D,13,train,1,1,1


### Encoding variables

In [189]:
categorical_columns

['family',
 'holiday_type',
 'holiday_locale',
 'locale_name',
 'holiday_transferred',
 'city',
 'state',
 'store_type',
 'store_cluster']

In [190]:
combined = pd.get_dummies(combined, columns=categorical_columns, drop_first=True)
print(list(combined.columns))

['Unnamed: 0', 'id', 'date', 'store_nbr', 'sales', 'onpromotion', 'daily_oil_price', 'dataset', 'day_of_week', 'month', 'quarter', 'family_BABY CARE', 'family_BEAUTY', 'family_BEVERAGES', 'family_BOOKS', 'family_BREAD/BAKERY', 'family_CELEBRATION', 'family_CLEANING', 'family_DAIRY', 'family_DELI', 'family_EGGS', 'family_FROZEN FOODS', 'family_GROCERY I', 'family_GROCERY II', 'family_HARDWARE', 'family_HOME AND KITCHEN I', 'family_HOME AND KITCHEN II', 'family_HOME APPLIANCES', 'family_HOME CARE', 'family_LADIESWEAR', 'family_LAWN AND GARDEN', 'family_LINGERIE', 'family_LIQUOR,WINE,BEER', 'family_MAGAZINES', 'family_MEATS', 'family_PERSONAL CARE', 'family_PET SUPPLIES', 'family_PLAYERS AND ELECTRONICS', 'family_POULTRY', 'family_PREPARED FOODS', 'family_PRODUCE', 'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD', 'holiday_type_Bridge', 'holiday_type_Event', 'holiday_type_Holiday', 'holiday_type_Transfer', 'holiday_type_Work Day', 'holiday_type_none', 'holiday_locale_National', 'holi

### Split back into train and test

In [191]:
train_processed = combined[combined['dataset'] == 'train'].drop(columns=['dataset'])
test_processed = combined[combined['dataset'] == 'test'].drop(columns=['dataset'])

## Modeling